In [1]:
import numpy as np
import pandas as pd
import string
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 400)

In [9]:
file_name = "Tab.delimited.Cleaned.dataset.WITH.variable.labels"
df = pd.read_table(r"./ML1/" + file_name + ".csv", sep='\t', encoding='ISO-8859-1', low_memory=False)

In [10]:
#Getting columns with one unique value, and the remaining are missing
single_valued_col_with_missing = []
for col in list(df):
  unique_values = df[col].unique()
  if unique_values.shape[0] == 2 and any(df[col].isnull()):
    single_valued_col_with_missing.append(col)

In [11]:
def preprocess_cols(df):  
  #Replace any useless values with NaN
  def check_replace(s):
    if type(s) != str:
      return s
    regex = string.punctuation + ' '
    all_special = all(i in regex for i in s)
    if all_special:
      return np.NaN
    else:
      return s.strip()

  for col in list(df):
    df[col] = df[col].apply(lambda x: check_replace(x))

  #Removing single valued columns
  single_value_cols = []
  for col in list(df):
    if df[col].unique().shape[0] == 1:
      single_value_cols.append(col)
  
  #Preprocessing of specific columns
  race_dict = {'White':'White', 
            'Black or African American':'Black or African American', 
            'East Asian':'East Asian', 
            'Other or Unknown':'Other or Unknown', 
            'South Asian':'South Asian', 
            'More than one race - Other':'More than one race - Other', 
            'turk':'turk', 
            'chinese':'East Asian', 
            'Nederlands':'Other or Unknown', 
            'More than one race - Black/White':'More than one race - Other', 
            'brazilwhite':'Other or Unknown', 
            'brazilbrown':'Other or Unknown', 
            'American Indian/Alaska Native':'Other or Unknown', 
            'Native Hawaiian or other Pacific Islander':'Other or Unknown', 
            'brazilblack':'Other or Unknown', 
            'brazilyellow':'Other or Unknown', 
            'indian':'South Asian', 
            'malay':'Other or Unknown', 
            'Nederlandse':'Other or Unknown', 
            'nederlands':'Other or Unknown', 
            'Belgisch Nederlands':'Other or Unknown', 
            'Marokkaans Nederlands':'Other or Unknown', 
            'brazilindigenous':'Other or Unknown', 
            'italiaans nederlands':'Other or Unknown', 
            'Turks Nederlands':'Other or Unknown', 
            'duits':'Other or Unknown', 
            'Russian':'Other or Unknown', 
            'nl':'Other or Unknown', 
            'Duits':'Other or Unknown'}

  df.race = df.race.apply(lambda x: x if pd.isnull(x) else race_dict[x])

  scalesa_dict = {'Up to a half hour': 1, 
                'Half an hour to an hour': 2, 
                'One to one and a half hours': 3, 
                'One and a half to two hours': 4, 
                'Two to two and a half hours': 5, 
                'More than two and a half hours': 6}

  df.scalesa = df.scalesa.apply(lambda x: x if pd.isnull(x) else scalesa_dict[x])

  scalesb_dict = {'Up to two and a half hours': 1, 
                 'Two and a half to three hours': 2, 
                 'Three to three and a half hours': 3, 
                 'Three and a half to four hours': 4, 
                 'Four to four and a half hours': 5, 
                 'More than four and a half hours': 6}

  df.scalesb = df.scalesb.apply(lambda x: x if pd.isnull(x) else scalesb_dict[x])

  noflagtimeestimate_dict = {'Morning': 1, 
                            'Afternoon': 2, 
                            'Evening': 3}

  df.noflagtimeestimate1 = df.noflagtimeestimate1.apply(lambda x: x if pd.isnull(x) else noflagtimeestimate_dict[x])
  df.noflagtimeestimate2 = df.noflagtimeestimate2.apply(lambda x: x if pd.isnull(x) else noflagtimeestimate_dict[x])
  df.noflagtimeestimate3 = df.noflagtimeestimate3.apply(lambda x: x if pd.isnull(x) else noflagtimeestimate_dict[x])
  df.noflagtimeestimate4 = df.noflagtimeestimate4.apply(lambda x: x if pd.isnull(x) else noflagtimeestimate_dict[x])
  
  
  #Any text in a language other than English has value 10, and different variations showing levels of preference
  #over art or math are encoded from 1 to 9
  text_dict = {
      'Your data suggest a strong preference for Arts compared to Mathematics.': 5,
      'Your data suggest a moderate preference for Arts compared to Mathematics.': 4,
      'Your data suggest little or no preference for Mathematics compared Arts.': 6,
      'Your data suggest a slight preference for Arts compared to Mathematics.': 3,
      'Your data suggest a slight preference for Mathematics compared to Arts.': 7,
      'There were too many errors made to determine a result.': 0,
      'Your data suggest a moderate preference for Mathematics compared to Arts.': 8,
      'There were too many fast trials to determine a result.': 1,
      'Your data suggest a strong preference for Mathematics compared to Arts.': 9
  }

  df.text = df.text.apply(lambda x: x if pd.isnull(x) else(10 if x not in text_dict.keys() else text_dict[x]))
  

  #Make Columns 155 to 162 uniform - (Integers) and perform one hot encoding.
  #There are 7 different values for each column, so I will take 7 bits.
  #For sysjust3 and sysjust7, 1 = strongly agree and 7 = strongly disagree
  #For sysjust(1, 2, 4, 5, 6, 8), 1 = strongly disagree and 7 = strongly agree
  i = 0
  for col in list(df):
      if i > 154 and i < 163:
          if i == 157 or i == 161:
              df = df.replace("Strongly agree", str(1))
              df = df.replace("Strongly disagree", str(7))
          else:
              df = df.replace("Strongly agree", str(7))
              df = df.replace("Strongly disagree", str(1))
      i = i + 1    

  for i in range(155, 163):
      data = df.iloc[:, i].tolist()
      testdata = ['1', '2', '3', '4', '5', '6', '7']

      numbers = '1234567'
      char_to_int = dict((c, i) for i, c in enumerate(numbers))
      int_to_char = dict((i, c) for i, c in enumerate(numbers))
      # integer encode input data
      integer_encoded = []
      #for num in testdata:
      for num in data:
          if not pd.isnull(num):
              integer_encoded.append(char_to_int[num])
          else:
              integer_encoded.append(num)
      #integer_encoded = [char_to_int[num] for num in testdata]
      #print(integer_encoded)

      #One hot encoding
      one_hot_encoded = list()
      for value in integer_encoded:
          if not pd.isnull(value):
              bit = [0 for _ in range(len(numbers))]
              bit[value] = 1
          else:
              bit = np.NaN
          one_hot_encoded.append(bit)

      concat_list = []
      for j in range(len(one_hot_encoded)):
          result = ''
          if type(one_hot_encoded[j]) != list:
              concat_list.append(np.NaN)
              continue
          for element in one_hot_encoded[j]:
              result = result + str(element)
          concat_list.append(result)    
          #concat_list.append(''.join(map(str,one_hot_encoded[i])))
      df1 = pd.DataFrame(concat_list)
#       print(df1)
      df.iloc[:, i] = df1[0]

  print(df.iloc[:, 155:163])
  
  #removing nlp columns (columns that have more than 35 unique values)
  nlp_cols = []
  i = 0
  for col in list(df):
    if i > 127:
      if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        i = i + 1
        continue
      if len(df[col].unique()) > 35:
        nlp_cols.append(col)
    i = i + 1
  
  def assign_label(x):
    global unique
    global number
    if pd.isnull(x):
      return np.NaN
    if x in unique.keys():
      return unique[x]
    else:
      unique[x] = number
      number = number + 1
      return (number - 1)

  exclude_cols = ['scalesa', 'scalesb', 'noflagtimeestimate1', 'noflagtimeestimate2', 'noflagtimeestimate3', \
                 'noflagtimeestimate4', 'text']
  exclude_cols.extend(["sysjust" + str(i) for i in range(1, 9)])
  i = 0
  for col in list(df):
    if col in exclude_cols or i < 127:
      i = i + 1
      continue
    global unique
    global number
    unique = {}
    number = 0
    df[col] = df[col].apply(lambda x: assign_label(x))  

  remove_cols = ['user_id', 'previous_session_id', 'previous_session_schema', 'user_agent', 'citizenship2', \
                'mturk.non.US', 'mturk.exclude']
  remove_cols.extend(["task_url." + str(i) for i in range(46)])
  remove_cols.extend(["task_creation_date." + str(i) for i in range(46)])

  # temporarily removed columns, can be considered later
  remove_cols.extend(['religion'])
  remove_cols.extend(["task_id." + str(i) for i in range(46)])

  remove_cols.extend(single_value_cols)
  remove_cols.extend(nlp_cols)

  remove_cols = list(set(remove_cols))
  df = df.drop(remove_cols, axis=1)
  
  #Saving and loading modified dataframe
  df.to_csv(r"ML1/" + file_name + ".preprocessed1" + ".csv", sep=',', encoding='utf-8', index=False)
  return df

In [12]:
df = preprocess_cols(df)

     sysjust1 sysjust2 sysjust3 sysjust4 sysjust5 sysjust6 sysjust7 sysjust8
0     1000000  0001000  0010000  0001000  0001000  0001000  0000001  1000000
1     0100000  0100000  0001000  0100000  0001000  0000001  0100000  0001000
2     0010000  0001000  0010000  0001000  0000100  0000001  0001000  0001000
3     0000100  0100000  0010000  0001000  0010000  1000000  0010000  0001000
4     0000100  0100000  0010000  0000001  0000100  0010000  0000100  0100000
5     0000100  0000100  0000010  0000010  0010000  0000010  0001000  0010000
6     0010000  0010000  0001000  0001000  0000100  0000100  0000100  0000001
7     0000010  0001000  0010000  0001000  0100000  0000100  0010000  0000010
8     1000000  1000000  0000001  0000010  0000001  0000001  0000001  0000010
9     1000000  1000000  0000001  1000000  0010000  1000000  0000001  0000100
...       ...      ...      ...      ...      ...      ...      ...      ...
6334  0100000  0001000  0000001  0000100  0000010  1000000  0100000  1000000

In [17]:
df.iloc[:, 100:]

,flagtimeestimate2,flagtimeestimate3,flagtimeestimate4,gamblerfallacya,gamblerfallacyb,iatexplicitart1,iatexplicitart2,iatexplicitart3,iatexplicitart4,iatexplicitart5,iatexplicitart6,iatexplicitmath1,iatexplicitmath2,iatexplicitmath3,iatexplicitmath4,iatexplicitmath5,iatexplicitmath6,imagineddescribe,imaginedexplicit1,imaginedexplicit2,imaginedexplicit3,imaginedexplicit4,imptaskto,major,mathwarm,moneyagea,moneygendera,moneygenderb,nativelang,noflagtimeestimate1,noflagtimeestimate2,noflagtimeestimate3,noflagtimeestimate4,omdimc3,politicalid,quotea,quoteb,race,reciprocityothera,reciprocityotherb,reciprocityusa,reciprocityusb,scalesa,scalesb,sex,sunkcosta,sunkcostb,sysjust1,sysjust2,sysjust3,sysjust4,sysjust5,sysjust6,sysjust7,sysjust8,text,study_url,study_name,us_or_international,lab_or_online,priorexposure1,priorexposure10,priorexposure11,priorexposure12,priorexposure13,priorexposure2,priorexposure3,priorexposure4,priorexposure5,priorexposure6,priorexposure7,priorexposure8,priorexposure9,mturk.Submitted.PaymentReq,mturk.total.mini.exps,mturk.duplicate,mturk.exclude.null,mturk.keep,filter_$,order,block2_meanerror,block5_meanerror,iat_exclude,o1,o2,o3,o4,o5,o6,o7,o8,o9,o10,o11,scalesorder,reciprocorder,diseaseforder,quoteorder,flagprimorder,sunkcostorder,anchorinorder,allowedforder,gamblerforder,moneypriorder,imaginedorder
0,NaN,NaN,NaN,NaN,3.16227766016838,6,4,7,6,5,5,6,6,6,5,5,5,This is interesting. I never knew that. I wond...,9,4,6,9,NaN,10,0,NaN,NaN,0.0,0.0,1.0,3.0,2.0,3.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,1.0,0.0,0.0,NaN,1000000,0001000,0010000,0001000,0001000,0001000,0000001,1000000,4.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Evening,Afternoon,Evening,NaN,6,4,4,4,Moderately Disgusting,4,4,Moderately bad,4,5,4,Very Avoid,5,I imagined meeting the person on a street. She...,5,5,1,5,NaN,13,55,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,1.0,NaN,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,0100000,0100000,0001000,0100000,0001000,0000001,0100000,0001000,4.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,1.0,1.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,Morning,Morning,Evening,2.23606797749979,NaN,6,6,6,6,5,7,Slightly bad,Moderately Sad,4,4,Very Avoid,Very Afraid,There is a beach with clean white sand. Tall p...,5,3,5,1,NaN,13,72,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,2.0,NaN,1.0,NaN,0.0,1.0,NaN,0.0,NaN,0.0,0010000,0001000,0010000,0001000,0000100,0000001,0001000,0001000,3.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0,2,2,2,1,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,1,0
3,Evening,Afternoon,Evening,NaN,2,5,5,6,5,5,4,5,4,4,4,5,4,"mountains, forrest, wildlife, waterfalls, wint...",4,2,5,7,NaN,2,40,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,2.0,NaN,2.0,NaN,1.0,NaN,0.0,NaN,1.0,1.0,1.0,NaN,0000100,0100000,0010000,0001000,0010000,1000000,0010000,0001000,5.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,2.0,2.0,0,1,3,3,2,0,1,1,2,3,3,2,0,3,1,2,3,1,3,3,1,2,2
4,NaN,NaN,NaN,3.46410161513775,NaN,4,Very Sad,6,4,Very Avoid,4,5,5,5,5,5,5,I imagined myself walking on a deserted boardw...,3,1,1,1,NaN,7,0,0.0,0.0,NaN,0.0,1.0,2.0,1.0,3.0,1.0,3.0,3.0,NaN,2.0,NaN,1.0,NaN,1.0,NaN,1.0,1.0,0.0,NaN,0000100,0100000,0010000,0000001,0000100,0010000,0000100,0100000,6.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,0.0,3.0,0,0,3,0,3,3,1,3,3,3,4,3,3,4,3,0,4,1,3,4,3,0,2
5,NaN,NaN,NaN,NaN,2.23606797749979,5,4,6,5,6,4,Slightly bad,Moderately Sad,4,Slightly Disgusting,Moderately Avoid,Moderately Afraid,I notice other people there as well; jogging a...,3,2,5,9,NaN,7,66,0.0,1.0,NaN,0.0,1.0,2.0,1.0,2.0,1.0,1.0,4.0,NaN,1.0,0.0,NaN,0.0,NaN,3.0,NaN,0.0,NaN,1.0,0000100,0000100,0000010,0000010,0010000,0000010,0001000,0010000,0.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN